# Model with Binary Data

Example testing a model which uses binary data
 

In [1]:
!cp ../../../proto/prediction.proto ./proto
!cp -r ../../../proto/tensorflow/tensorflow .
!python -m grpc.tools.protoc -I. --python_out=. --grpc_python_out=. ./proto/prediction.proto

In [2]:
import json
import requests
import base64
from proto import prediction_pb2
from proto import prediction_pb2_grpc
import grpc
import numpy as np
import pickle

def get_payload():
    array_2d = np.arange(9).reshape(3, 3)
    print(array_2d)
    bdata = pickle.dumps(array_2d)
    return bdata

In [17]:
def rest_request_ambassador(deploymentName,request,endpoint="localhost:8003"):
    response = requests.post(
                "http://"+endpoint+"/seldon/"+deploymentName+"/api/v0.1/predictions",
                json=request)
    print(response.status_code)
    print(response.text)
    return response.json()

def grpc_request_ambassador(deploymentName,data,endpoint="localhost:8004"):
    request = prediction_pb2.SeldonMessage(binData = data)
    channel = grpc.insecure_channel(endpoint)
    stub = prediction_pb2_grpc.SeldonStub(channel)
    metadata = [('seldon',deploymentName)]
    response = stub.Predict(request=request,metadata=metadata)
    return response

def rest_request_docker(request,endpoint="localhost:5000"):
    response = requests.post(
                "http://"+endpoint+"/predict",
                 data={"json":json.dumps(request),"isDefault":True})
    print(response.text)
    return response.json()

def grpc_request_docker(data,endpoint="localhost:5000"):
    request = prediction_pb2.SeldonMessage(binData = data)
    channel = grpc.insecure_channel(endpoint)
    stub = prediction_pb2_grpc.ModelStub(channel)
    response = stub.Predict(request=request)
    return response


# REST

In [60]:
!s2i build -E environment_rest . seldonio/seldon-core-s2i-python36:0.5.1-SNAPSHOT model-with-bindata-rest:0.1

---> Installing application source...
Build completed successfully


In [61]:
!docker run --name "model-with-bindata" -d --rm -p 5000:5000 model-with-bindata-rest:0.1

ffe70f6aea345e9eeab52842c5ada67fbc5cf5a22cc638ce162e96539d4b7d5f


In [65]:
bdata = get_payload()
bdata_base64 = base64.b64encode(bdata).decode('utf-8')
payload = {"meta":{},"binData":bdata_base64}
response = rest_request_docker(payload)
bdata2 = base64.b64decode(response["binData"])
arr_resp = pickle.loads(bdata2)
print(arr_resp)
print(arr_resp.shape)

[[0 1 2]
 [3 4 5]
 [6 7 8]]
{"binData":"gANjbnVtcHkuY29yZS5tdWx0aWFycmF5Cl9yZWNvbnN0cnVjdApxAGNudW1weQpuZGFycmF5CnEBSwCFcQJDAWJxA4dxBFJxBShLAUsDSwOGcQZjbnVtcHkKZHR5cGUKcQdYAgAAAGk4cQhLAEsBh3EJUnEKKEsDWAEAAAA8cQtOTk5K/////0r/////SwB0cQxiiUNIAAAAAAAAAAABAAAAAAAAAAIAAAAAAAAAAwAAAAAAAAAEAAAAAAAAAAUAAAAAAAAABgAAAAAAAAAHAAAAAAAAAAgAAAAAAAAAcQ10cQ5iLg==","meta":{"metrics":[{"key":"mycounter","type":"COUNTER","value":1},{"key":"mygauge","type":"GAUGE","value":100},{"key":"mytimer","type":"TIMER","value":20.2}]}}

[[0 1 2]
 [3 4 5]
 [6 7 8]]
(3, 3)


In [66]:
!docker rm model-with-bindata --force

model-with-bindata


# gRPC

In [22]:
!s2i build -E environment_grpc . seldonio/seldon-core-s2i-python36:0.5.1-SNAPSHOT model-with-bindata-grpc:0.1

---> Installing application source...
Build completed successfully


In [23]:
!docker run --name "model-with-bindata" -d --rm -p 5000:5000 model-with-bindata-grpc:0.1

02365f08b326f43f72576d20b9cfb4ed71c1e404edaccbe7396d98e5bef7e30e


In [27]:
payload = get_payload()

In [58]:
bdata = get_payload()
resp = grpc_request_docker(bdata)
bdata2 = resp.binData
arr_resp = pickle.loads(bdata2)
print(arr_resp)
print(arr_resp.shape)

[[0 1 2]
 [3 4 5]
 [6 7 8]]
[[0 1 2]
 [3 4 5]
 [6 7 8]]
(3, 3)


In [59]:
!docker rm model-with-bindata --force

model-with-bindata


# Test using Minikube

**Due to a [minikube/s2i issue](https://github.com/SeldonIO/seldon-core/issues/253) you will need [s2i >= 1.1.13](https://github.com/openshift/source-to-image/releases/tag/v1.1.13)**

In [68]:
!minikube start --vm-driver kvm2 --memory 4096 --feature-gates=CustomResourceValidation=true --extra-config=apiserver.Authorization.Mode=RBAC

Starting local Kubernetes v1.10.0 cluster...
Starting VM...
Getting VM IP address...
Moving files into cluster...
Setting up certs...
Connecting to cluster...
Setting up kubeconfig...
Starting cluster components...
^C


In [69]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

clusterrolebinding.rbac.authorization.k8s.io/kube-system-cluster-admin created


In [70]:
!helm init

$HELM_HOME has been configured at /home/clive/.helm.

Tiller (the Helm server-side component) has been installed into your Kubernetes Cluster.

Please note: by default, Tiller is deployed with an insecure 'allow unauthenticated users' policy.
To prevent this, run `helm init` with the --tiller-tls-verify flag.
For more information on securing your installation see: https://docs.helm.sh/using_helm/#securing-your-helm-installation
Happy Helming!


In [71]:
!kubectl rollout status deploy/tiller-deploy -n kube-system

Waiting for deployment "tiller-deploy" rollout to finish: 0 of 1 updated replicas are available...
deployment "tiller-deploy" successfully rolled out


In [72]:
!helm install ../../../helm-charts/seldon-core-crd --name seldon-core-crd  --set usage_metrics.enabled=true
!helm install ../../../helm-charts/seldon-core --name seldon-core --set ambassador.enabled=true

NAME:   seldon-core-crd
LAST DEPLOYED: Fri Nov 30 19:34:07 2018
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1beta1/Deployment
NAME                        DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
seldon-spartakus-volunteer  1        0        0           0          0s

==> v1/ServiceAccount
NAME                        SECRETS  AGE
seldon-spartakus-volunteer  1        0s

==> v1beta1/ClusterRole
NAME                        AGE
seldon-spartakus-volunteer  0s

==> v1beta1/ClusterRoleBinding
NAME                        AGE
seldon-spartakus-volunteer  0s

==> v1/ConfigMap
NAME                     DATA  AGE
seldon-spartakus-config  3     0s

==> v1beta1/CustomResourceDefinition
NAME                                         AGE
seldondeployments.machinelearning.seldon.io  0s


NOTES:
NOTES: TODO


NAME:   seldon-core
LAST DEPLOYED: Fri Nov 30 19:34:07 2018
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1beta1/RoleBinding
NAME        AGE
ambassador  1s

==> v1beta1/ClusterRole


# REST

In [20]:
!eval $(minikube docker-env) && s2i build -E environment_rest . seldonio/seldon-core-s2i-python36:0.5.1-SNAPSHOT model-with-bindata-rest:0.1

---> Installing application source...
Build completed successfully


In [76]:
!kubectl create -f deployment-rest.json

seldondeployment.machinelearning.seldon.io/mymodel created


Wait until ready (replicas == replicasAvailable)

In [77]:
!kubectl get seldondeployments mymodel -o jsonpath='{.status}' 

map[predictorStatus:[map[name:mymodel-mymodel-b0e3779 replicas:1 replicasAvailable:0]] state:Creating]

## Test predict

In [4]:
minikube_ip = !minikube ip
minikube_port = !kubectl get svc -l app.kubernetes.io/name=ambassador -o jsonpath='{.items[0].spec.ports[0].nodePort}'

In [6]:
bdata = get_payload()
bdata_base64 = base64.b64encode(bdata).decode('utf-8')
payload = {"meta":{},"binData":bdata_base64}
response = rest_request_ambassador("mymodel",payload,minikube_ip[0]+":"+minikube_port[0])
bdata2 = base64.b64decode(response["binData"])
arr_resp = pickle.loads(bdata2)
print(arr_resp)
print(arr_resp.shape)

[[0 1 2]
 [3 4 5]
 [6 7 8]]
200
{
  "meta": {
    "puid": "n0jtefhl4dbo8v25bntnkjuf03",
    "tags": {
    },
    "routing": {
    },
    "requestPath": {
      "complex-model": "model-with-bindata-rest:0.1"
    },
    "metrics": [{
      "key": "mycounter",
      "type": "COUNTER",
      "value": 1.0,
      "tags": {
      }
    }, {
      "key": "mygauge",
      "type": "GAUGE",
      "value": 100.0,
      "tags": {
      }
    }, {
      "key": "mytimer",
      "type": "TIMER",
      "value": 20.2,
      "tags": {
      }
    }]
  },
  "binData": "gANjbnVtcHkuY29yZS5tdWx0aWFycmF5Cl9yZWNvbnN0cnVjdApxAGNudW1weQpuZGFycmF5CnEBSwCFcQJDAWJxA4dxBFJxBShLAUsDSwOGcQZjbnVtcHkKZHR5cGUKcQdYAgAAAGk4cQhLAEsBh3EJUnEKKEsDWAEAAAA8cQtOTk5K/////0r/////SwB0cQxiiUNIAAAAAAAAAAABAAAAAAAAAAIAAAAAAAAAAwAAAAAAAAAEAAAAAAAAAAUAAAAAAAAABgAAAAAAAAAHAAAAAAAAAAgAAAAAAAAAcQ10cQ5iLg=="
}
[[0 1 2]
 [3 4 5]
 [6 7 8]]
(3, 3)


In [7]:
!kubectl delete -f deployment-rest.json

seldondeployment.machinelearning.seldon.io "mymodel" deleted


# gRPC

In [19]:
!eval $(minikube docker-env) && s2i build -E environment_grpc . seldonio/seldon-core-s2i-python36:0.5.1-SNAPSHOT model-with-bindata-grpc:0.1

---> Installing application source...
Build completed successfully


In [9]:
!kubectl create -f deployment-grpc.json

seldondeployment.machinelearning.seldon.io/mymodel created


Wait until ready (replicas == replicasAvailable)

In [11]:
!kubectl get seldondeployments mymodel -o jsonpath='{.status}' 

map[predictorStatus:[map[name:mymodel-mymodel-a9ecaa4 replicas:1 replicasAvailable:1]] state:Available]

In [18]:
bdata = get_payload()
response = grpc_request_ambassador("mymodel",bdata,minikube_ip[0]+":"+minikube_port[0])
bdata2 = response.binData
arr_resp = pickle.loads(bdata2)
print(arr_resp)
print(arr_resp.shape)

[[0 1 2]
 [3 4 5]
 [6 7 8]]
[[0 1 2]
 [3 4 5]
 [6 7 8]]
(3, 3)


In [ ]:
!kubectl delete -f deployment-grpc.json

In [ ]:
!minikube delete